# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9e88847160>
├── 'a' --> tensor([[-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594]])
└── 'x' --> <FastTreeValue 0x7f9e88847070>
    └── 'c' --> tensor([[ 0.9274,  1.2868,  0.0517, -0.3390],
                        [ 1.5586,  0.1712,  1.1923, -0.6800],
                        [-0.0963, -0.1006,  0.3206, -1.0464]])

In [4]:
t.a

tensor([[-1.3231,  0.0383,  0.0237],
        [-0.5559, -0.9390,  0.3594]])

In [5]:
%timeit t.a

63.9 ns ± 0.0189 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9e88847160>
├── 'a' --> tensor([[-0.8165, -0.3284,  0.0015],
│                   [-0.8182, -0.4041, -0.0124]])
└── 'x' --> <FastTreeValue 0x7f9e88847070>
    └── 'c' --> tensor([[ 0.9274,  1.2868,  0.0517, -0.3390],
                        [ 1.5586,  0.1712,  1.1923, -0.6800],
                        [-0.0963, -0.1006,  0.3206, -1.0464]])

In [7]:
%timeit t.a = new_value

68.5 ns ± 0.0579 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3231,  0.0383,  0.0237],
               [-0.5559, -0.9390,  0.3594]]),
    x: Batch(
           c: tensor([[ 0.9274,  1.2868,  0.0517, -0.3390],
                      [ 1.5586,  0.1712,  1.1923, -0.6800],
                      [-0.0963, -0.1006,  0.3206, -1.0464]]),
       ),
)

In [10]:
b.a

tensor([[-1.3231,  0.0383,  0.0237],
        [-0.5559, -0.9390,  0.3594]])

In [11]:
%timeit b.a

54.4 ns ± 0.0412 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2632, -0.6425, -0.1998],
               [ 0.1101,  1.3349,  0.4263]]),
    x: Batch(
           c: tensor([[ 0.9274,  1.2868,  0.0517, -0.3390],
                      [ 1.5586,  0.1712,  1.1923, -0.6800],
                      [-0.0963, -0.1006,  0.3206, -1.0464]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.426 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

855 ns ± 0.148 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 423 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 312 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9ddc52aca0>
├── 'a' --> tensor([[[-1.3231,  0.0383,  0.0237],
│                    [-0.5559, -0.9390,  0.3594]],
│           
│                   [[-1.3231,  0.0383,  0.0237],
│                    [-0.5559, -0.9390,  0.3594]],
│           
│                   [[-1.3231,  0.0383,  0.0237],
│                    [-0.5559, -0.9390,  0.3594]],
│           
│                   [[-1.3231,  0.0383,  0.0237],
│                    [-0.5559, -0.9390,  0.3594]],
│           
│                   [[-1.3231,  0.0383,  0.0237],
│                    [-0.5559, -0.9390,  0.3594]],
│           
│                   [[-1.3231,  0.0383,  0.0237],
│                    [-0.5559, -0.9390,  0.3594]],
│           
│                   [[-1.3231,  0.0383,  0.0237],
│                    [-0.5559, -0.9390,  0.3594]],
│           
│                   [[-1.3231,  0.0383,  0.0237],
│                    [-0.5559, -0.9390,  0.3594]]])
└── 'x' --> <FastTreeValue 0x7f9ddc52a580>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.4 µs ± 82.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9ddc4cdf40>
├── 'a' --> tensor([[-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594],
│                   [-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594],
│                   [-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594],
│                   [-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594],
│                   [-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594],
│                   [-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594],
│                   [-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594],
│                   [-1.3231,  0.0383,  0.0237],
│                   [-0.5559, -0.9390,  0.3594]])
└── 'x' --> <FastTreeValue 0x7f9ddc4ccf40>
    └── 'c' --> tensor([[ 0.9274,  1.2868,  0.0517, -0.3390],
                        [ 1.5586,  0.1712,  1.1923, -0.6800],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58 µs ± 924 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.9274,  1.2868,  0.0517, -0.3390],
                       [ 1.5586,  0.1712,  1.1923, -0.6800],
                       [-0.0963, -0.1006,  0.3206, -1.0464]],
              
                      [[ 0.9274,  1.2868,  0.0517, -0.3390],
                       [ 1.5586,  0.1712,  1.1923, -0.6800],
                       [-0.0963, -0.1006,  0.3206, -1.0464]],
              
                      [[ 0.9274,  1.2868,  0.0517, -0.3390],
                       [ 1.5586,  0.1712,  1.1923, -0.6800],
                       [-0.0963, -0.1006,  0.3206, -1.0464]],
              
                      [[ 0.9274,  1.2868,  0.0517, -0.3390],
                       [ 1.5586,  0.1712,  1.1923, -0.6800],
                       [-0.0963, -0.1006,  0.3206, -1.0464]],
              
                      [[ 0.9274,  1.2868,  0.0517, -0.3390],
                       [ 1.5586,  0.1712,  1.1923, -0.6800],
                       [-0.0963, -0.1006,  0.3206, -1.0464]],

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 265 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.9274,  1.2868,  0.0517, -0.3390],
                      [ 1.5586,  0.1712,  1.1923, -0.6800],
                      [-0.0963, -0.1006,  0.3206, -1.0464],
                      [ 0.9274,  1.2868,  0.0517, -0.3390],
                      [ 1.5586,  0.1712,  1.1923, -0.6800],
                      [-0.0963, -0.1006,  0.3206, -1.0464],
                      [ 0.9274,  1.2868,  0.0517, -0.3390],
                      [ 1.5586,  0.1712,  1.1923, -0.6800],
                      [-0.0963, -0.1006,  0.3206, -1.0464],
                      [ 0.9274,  1.2868,  0.0517, -0.3390],
                      [ 1.5586,  0.1712,  1.1923, -0.6800],
                      [-0.0963, -0.1006,  0.3206, -1.0464],
                      [ 0.9274,  1.2868,  0.0517, -0.3390],
                      [ 1.5586,  0.1712,  1.1923, -0.6800],
                      [-0.0963, -0.1006,  0.3206, -1.0464],
                      [ 0.9274,  1.2868,  0.0517, -0.3390],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 380 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 6.16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
